In [12]:
!pwd

/jet/home/wlu1/GPCR_LigandGen


In [13]:
import os
import pandas as pd
import numpy as np
import py3Dmol
from rdkit import Chem
from rdkit.Chem import AllChem, rdMolAlign, rdFMCS
import sys
import math
from rdkit import Chem
from posebusters import PoseBusters

In [14]:
# run posebusters on saved sdf files
gen_ligands_sdf_file = "data/generated_ligands/generated_ligands.sdf"
# ref_ligand_sdf_file = "data/pockets/mrgx4_human_8K4S_ligand_JW0.sdf"
# eventually do with ref ligand, conditioning pocket

buster = PoseBusters(config="mol")
# posebuster_df = buster.bust([gen_ligands_sdf_file], ref_ligand_sdf_file, None, full_report=True)
posebuster_df = buster.bust([gen_ligands_sdf_file], None, None, full_report=True)
# df = buster.bust(gen_ligand_mols, ref_ligand_mol, None, full_report=True)

posebuster_df.to_csv("data/metrics/posebusters_results_pred_mol.csv", index=False)
print(posebuster_df.shape)
display(posebuster_df)

[14:36:15] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 8 10 11 12 13 14 15 16
[14:36:19] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 8 18 20 24 27
[14:36:20] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 7 8 9 11 12
[14:36:20] Can't kekulize mol.  Unkekulized atoms: 0 2 4 5 6 11 12 15 16
[14:36:21] Can't kekulize mol.  Unkekulized atoms: 0 1 2 8 10
[14:36:21] Can't kekulize mol.  Unkekulized atoms: 0 1 3 4 5 6 7 8 9 10 12
[14:36:21] Explicit valence for atom # 7 N, 4, is greater than permitted
[14:36:21] Explicit valence for atom # 7 N, 4, is greater than permitted
[14:36:21] Explicit valence for atom # 7 N, 4, is greater than permitted
[14:36:21] Explicit valence for atom # 7 N, 4, is greater than permitted
[14:36:21] Explicit valence for atom # 7 N, 4, is greater than permitted
[14:36:21] Explicit valence for atom # 7 N, 4, is greater than permitted
[14:36:21] Explicit valence for atom # 7 N, 4, is greater than permitted
[14:36:21] Can't kekulize mol.  Unkekulized ato

(64, 44)


mol_pred_loaded  \
file                                         molecule position                    
data/generated_ligands/generated_ligands.sdf          0                    True   
                                                      1                    True   
                                                      2                    True   
                                                      3                    True   
                                                      4                    True   
...                                                                         ...   
                                                      59                   True   
                                                      60                   True   
                                                      61                   True   
                                                      62                   True   
                                                      63                   True   

                                                                sanitization  \
file                                         molecule position                 
data/generated_ligands/generated_ligands.sdf          0                 True   
                                                      1                False   
                                                      2                 True   
                                                      3                 True   
                                                      4                 True   
...                                                                      ...   
                                                      59                True   
                                                      60                True   
                                                      61                True   
                                                      62                True   
                                                      63                True   

                                                                inchi_convertible  \
file                                         molecule position                      
data/generated_ligands/generated_ligands.sdf          0                      True   
                                                      1                     False   
                                                      2                      True   
                                                      3                      True   
                                                      4                      True   
...                                                                           ...   
                                                      59                     True   
                                                      60                     True   
                                                      61                     True   
                                                      62                     True   
                                                      63                     True   

                                                                all_atoms_connected  \
file                                         molecule position                        
data/generated_ligands/generated_ligands.sdf          0                        True   
                                                      1                        True   
                                                      2                        True   
                                                      3                        True   
                                                      4                        True   
...                                                                             ...   
                                                      59                       True   
                                                      

In [15]:
# filter things
# filter out invalid mols- mol loaded false, not inchi convertible, not all atoms connected, bond lengths, bond angles, steric clash, aromatic ring flatness- get indices
# filter stuff for non-ideal- no radicals, non-aromatic ring flatness, 
# sanitized?

posebuster_df['passed_filters'] = (
    (posebuster_df['sanitization'] == 1) &
    (posebuster_df['inchi_convertible'] == 1) &
    (posebuster_df['all_atoms_connected'] == 1) &
    (posebuster_df['no_radicals'] == 1) &
    (posebuster_df['bond_lengths'] == 1) &
    (posebuster_df['bond_angles'] == 1) &
    (posebuster_df['internal_steric_clash'] == 1) &
    (posebuster_df['aromatic_ring_flatness'] == 1) &
    (posebuster_df['non-aromatic_ring_non-flatness'] == 1) &
    (posebuster_df['double_bond_flatness'] == 1) &
    (posebuster_df['internal_energy'] == 1) &
    (posebuster_df['passes_kekulization'] == 1) &
    (posebuster_df['no_radicals_before_sanitization'] == 1) &
    (posebuster_df['number_clashes'] == 0)
)

# count passed molecules
print(posebuster_df['passed_filters'].sum())

# filtered_posebuster_df = posebuster_df[posebuster_df['passed_filters']]
# print(len(filtered_posebuster_df))

49


In [16]:
# get metrics from data/metrics/metrics.csv

metrics_path = os.path.join("data", "metrics", "metrics.csv")
metrics_df = pd.read_csv(metrics_path)
print(metrics_df.shape)

metrics_df['passed posebusters'] = posebuster_df['passed_filters'].values
metrics_df['mol_pred_energy'] = posebuster_df['mol_pred_energy'].values
metrics_df['ensemble_avg_energy'] = posebuster_df['ensemble_avg_energy'].values
metrics_df['energy_ratio'] = posebuster_df['energy_ratio'].values

metrics_df.head()

(64, 112)


,representation.smiles,medchem.valid,medchem.connected,medchem.qed,medchem.sa,medchem.logp,medchem.lipinski,medchem.size,medchem.n_rotatable_bonds,clashes.passed_clash_score_ligands,...,geometry.C-C:O,geometry.O=C-O,geometry.C-O-N,ring_count.num_8_rings,geometry.C#C,geometry.C#C-C,passed posebusters,mol_pred_energy,ensemble_avg_energy,energy_ratio
0,Oc1ccc2cc3ccncc3nc2c1,True,True,0.561678,2.190962,2.4886,5.0,15,0.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,True,52.489737,32.058221,1.637325
1,Cc1cc2nc3c4ccccc4ccc3n2n1,False,NaN,NaN,NaN,NaN,NaN,17,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,False,715.281074,62.105507,11.517192
2,OCC1C2=Cc3cc(O)ccc3CC21,True,True,0.701928,3.777115,1.5700,5.0,14,1.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,False,998.653455,NaN,NaN
3,Fc1ccc(-c2ccncc2)cc1,True,True,0.645723,1.456452,2.8877,5.0,13,1.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,True,43.816148,28.033257,1.563006
4,Oc1ccc(-c2ccccc2)cc1,True,True,0.696938,1.225356,3.0592,5.0,13,1.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,True,41.354910,28.713736,1.440248


In [20]:
# Flag molecules that pass DrugFlow filtering criteria

metrics_df['passed_filters'] = (
    (metrics_df['reos'] == 1) &
    (metrics_df['chembl_ring_systems'] == 1) &
    (metrics_df['novel'] == 1)
)

# flag in metrics the molecules passing posebusters filtering

# count number of molecules with too high of energy ratio - greater than 100
print((metrics_df['energy_ratio'] > 100).sum())

# Subset filtered molecules

filtered_metrics_df = metrics_df[metrics_df['passed_filters']]

0


In [18]:
filtered_metrics_df

,representation.smiles,medchem.valid,medchem.connected,medchem.qed,medchem.sa,medchem.logp,medchem.lipinski,medchem.size,medchem.n_rotatable_bonds,clashes.passed_clash_score_ligands,...,geometry.O=C-O,geometry.C-O-N,ring_count.num_8_rings,geometry.C#C,geometry.C#C-C,passed posebusters,mol_pred_energy,ensemble_avg_energy,energy_ratio,passed_filters
3,Fc1ccc(-c2ccncc2)cc1,True,True,0.645723,1.456452,2.88770,5.0,13,1.0,True,...,NaN,NaN,NaN,NaN,NaN,True,43.816148,28.033257,1.563006,True
4,Oc1ccc(-c2ccccc2)cc1,True,True,0.696938,1.225356,3.05920,5.0,13,1.0,True,...,NaN,NaN,NaN,NaN,NaN,True,41.354910,28.713736,1.440248,True
9,Oc1cnccc1-c1ccc2cnccc2c1,True,True,0.688020,2.199987,3.00240,5.0,17,1.0,True,...,NaN,NaN,NaN,NaN,NaN,True,88.415923,41.174846,2.147329,True
17,Clc1ccc(-c2ccn[nH]2)cc1,True,True,0.714915,1.893474,2.73010,5.0,12,1.0,True,...,NaN,NaN,NaN,NaN,NaN,True,65.975374,50.227643,1.313527,True
18,Cn1cc(-c2ccc(F)cc2)cn1,True,True,0.651064,1.754651,2.22620,5.0,13,1.0,True,...,NaN,NaN,NaN,NaN,NaN,True,67.820695,53.480021,1.268150,True
23,Oc1ccc2cc(O)ccc2c1,True,True,0.620101,1.768665,2.25100,5.0,12,0.0,True,...,NaN,NaN,NaN,NaN,NaN,True,39.662218,22.976159,1.726234,True
25,NCCCc1ccccc1,True,True,0.668660,1.342663,1.57790,5.0,10,3.0,True,...,NaN,NaN,NaN,NaN,NaN,True,60.075830,15.811200,3.799574,True
26,c1ccc(CCn2cccn2)cc1,True,True,0.693089,1.687078,2.12580,5.0,13,3.0,False,...,NaN,NaN,NaN,NaN,NaN,True,367.712940,50.267470,7.315127,True
34,N#Cc1cc2ccccc2[nH]1,True,True,0.600255,2.367754,2.03958,5.0,11,0.0,True,...,NaN,NaN,NaN,NaN,NaN,True,86.400524,49.520741,1.744734,True
37,Oc1ccc2cc(O)ccc2c1,True,True,0.620101,1.768665,2.25100,5.0,12,0.0,True,...,NaN,NaN,NaN,NaN,NaN,True,33.690163,22.976159,1.466310,True


In [19]:
# get indices of filtered molecules in arrays

filtered_ligand_indices = filtered_metrics_df.index.tolist()

# read data/gnina_output/gnina_summary.csv
gnina_summary_path = os.path.join("data", "gnina_output", "gnina_summary.csv")
gnina_summary_df = pd.read_csv(gnina_summary_path)

FileNotFoundError: [Errno 2] No such file or directory: 'data/gnina_output/gnina_summary.csv'

In [ ]:
filtered_gnina_metrics = gnina_summary_df.iloc[filtered_ligand_indices]

# sort ligands by highest vina_affinity
filtered_gnina_metrics = filtered_gnina_metrics.sort_values(by='vina_affinity', ascending=True)
filtered_gnina_metrics

,ligand,vina_affinity,cnn_pose_score,cnn_affinity
9,ligand_10,-6.31,0.6009,4.667
60,ligand_61,-6.23,0.5817,4.801
4,ligand_5,-6.16,0.6823,3.880
3,ligand_4,-5.92,0.6469,4.140
41,ligand_42,-5.66,0.7031,3.359
26,ligand_27,-5.56,0.6739,3.449
18,ligand_19,-5.38,0.6457,4.175
37,ligand_38,-5.35,0.6240,3.698
62,ligand_63,-5.23,0.6357,3.785
23,ligand_24,-5.17,0.6467,3.765


In [ ]:
def visualize_docked_molecule(ligand_index): 
    ligand_index = ligand_index + 1 # adjust for 1-based indexing in file names 
    receptor_path = os.path.join("data", "raw", "mrgx4_human_8K4S.pdb") 
    ligand_path = os.path.join("data", "generated_ligands", f"ligand_{ligand_index}.pdb") 
    docking_path = os.path.join("data", "gnina_output", f"ligand_{ligand_index}_docked.sdf") 
    
    v = py3Dmol.view() 
    v.addModel(open(receptor_path).read()) 
    v.setStyle({'cartoon':{},'stick':{'radius':.1}}) 

    #v.addModel(open(ligand_path).read()) 
    #v.setStyle({'model':1},{'stick':{'colorscheme':'dimgrayCarbon','radius':.125}}) 

    first_model = open(docking_path).read().split('OpenBabel12022519363D')[0]
    
   # v.addModelsAsFrames(open(docking_path).read()) 
    v.addModel(first_model)

    v.setStyle({'model':1},{'stick':{'colorscheme':'greenCarbon'}}) 
    v.zoomTo({'model':0}) 
    v.rotate(90) 
    #v.animate({'interval':1000, 'loop':'forward'}) 
    return v

In [ ]:
def compute_rmsd(ligand_index, remove_hs=True):

    ref_path = 'data/pockets/mrgx4_human_8K4S_ligand_JW0.pdb'   
    dock_sdf = f'data/gnina_output/ligand_{ligand_index + 1}_docked.sdf'
    remove_hs = True # set False if you want to include Hydrogens

    def load_ref(path):
        if path.lower().endswith(".pdb") or path.lower().endswith(".pdbqt"):
            mol = Chem.MolFromPDBFile(path, removeHs=remove_hs, sanitize=False)
            if mol:
                Chem.SanitizeMol(mol)
            return mol
        else:
            mol = Chem.MolFromMolFile(path, removeHs=remove_hs, sanitize=False)
            if mol:
                Chem.SanitizeMol(mol)
            return mol

    def sanitize_try(m):
        try:
            Chem.SanitizeMol(m)
            return True
        except Exception:
            try:
                Chem.Kekulize(m, clearAromaticFlags=True)
                Chem.SanitizeMol(m)
                return True
            except Exception:
                return False

    ref = load_ref(ref_path)
    if ref is None:
        print("ERROR: Could not read reference:", ref_path); sys.exit(1)
    ref = Chem.AddHs(ref) if not remove_hs else Chem.RemoveHs(ref)
    if not sanitize_try(ref):
        print("ERROR: Reference failed sanitization"); sys.exit(1)

    suppl = Chem.SDMolSupplier(dock_sdf, removeHs=False)
    out = []
    i = 0
    for dock_m in suppl:
        i += 1
        if dock_m is None:
            print(f"Pose {i}: could not read, skipping")
            out.append((i, float('nan')))
            continue

        # sanitize / normalize
        if not sanitize_try(dock_m):
            print(f"Pose {i}: sanitization failed, skipping")
            out.append((i, float('nan')))
            continue

        dock_proc = Chem.AddHs(dock_m) if not remove_hs else Chem.RemoveHs(Chem.Mol(dock_m))
        # exact atom count match?
        if ref.GetNumAtoms() == dock_proc.GetNumAtoms():
            # do maximum alignment by atom index (assumes same ordering)
            try:
                # compute alignment and RMSD
                rms = rdMolAlign.AlignMol(dock_proc, ref, atomMap=None)
                # AlignMol returns RMSD when it succeeds; if not, use GetBestRMS
                if math.isfinite(rms):
                    out.append((i, rms))
                    print(f"Pose {i}: RMSD (index-based) = {rms:.3f} Å")
                    continue
            except Exception:
                pass

        # try to find MCS and map atom indices
        mcs = rdFMCS.FindMCS([ref, dock_proc], atomCompare=rdFMCS.AtomCompare.CompareElements,
                            bondCompare=rdFMCS.BondCompare.CompareAny,
                            matchValences=False, ringMatchesRingOnly=True, completeRingsOnly=True,
                            timeout=10)
        if mcs.canceled or mcs.numAtoms == 0:
            print(f"Pose {i}: No MCS found. RMSD = NaN")
            out.append((i, float('nan')))
            continue

        # build substructure match lists
        patt = Chem.MolFromSmarts(mcs.smartsString)
        ref_matches = ref.GetSubstructMatches(patt)
        dock_matches = dock_proc.GetSubstructMatches(patt)
        if not ref_matches or not dock_matches:
            print(f"Pose {i}: No substructure matches found. RMSD = NaN")
            out.append((i, float('nan')))
            continue

        # choose first match mapping
        ref_map = ref_matches[0]
        dock_map = dock_matches[0]
        
        # create atomMap pairs (dock_idx, ref_idx)
        atomMap = list(zip(dock_map, ref_map))
        try:
            rms = rdMolAlign.AlignMol(dock_proc, ref, atomMap=atomMap)
            print(f"Pose {i}: RMSD (MCS-based) = {rms:.3f} Å (using {len(atomMap)} atoms)")
            out.append((i, rms))
        except Exception as e:
            print(f"Pose {i}: AlignMol failed: {e}")
            out.append((i, float('nan')))

    # summary
    print("\nSummary (pose, rmsd Å):")
    for item in out:
        print(item[0], item[1])

    return

In [ ]:
top_ligand_indices = filtered_gnina_metrics.index.tolist()[:5]
top_ligand_indices

[9, 60, 4, 3, 41]

In [ ]:
visualize_docked_molecule(top_ligand_indices[0])

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
compute_rmsd(ligand_index = top_ligand_indices[0])

Pose 1: RMSD (MCS-based) = 0.252 Å (using 6 atoms)
Pose 2: RMSD (MCS-based) = 0.252 Å (using 6 atoms)
Pose 3: RMSD (MCS-based) = 0.252 Å (using 6 atoms)
Pose 4: RMSD (MCS-based) = 0.252 Å (using 6 atoms)
Pose 5: RMSD (MCS-based) = 0.256 Å (using 6 atoms)
Pose 6: RMSD (MCS-based) = 0.252 Å (using 6 atoms)
Pose 7: RMSD (MCS-based) = 0.252 Å (using 6 atoms)
Pose 8: RMSD (MCS-based) = 0.252 Å (using 6 atoms)
Pose 9: RMSD (MCS-based) = 0.252 Å (using 6 atoms)

Summary (pose, rmsd Å):
1 0.25182691566997095
2 0.2518904243936733
3 0.25204937412382045
4 0.25180771973143656
5 0.2564835932938176
6 0.25173556016861937
7 0.2517994255271315
8 0.2519818499135111
9 0.2517299278397275


In [ ]:
visualize_docked_molecule(top_ligand_indices[1])

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
# Ligand is chemically invalid? RMSD cannot be computed

compute_rmsd(ligand_index = top_ligand_indices[1])

Pose 1: could not read, skipping
Pose 2: could not read, skipping
Pose 3: could not read, skipping
Pose 4: could not read, skipping
Pose 5: could not read, skipping
Pose 6: could not read, skipping
Pose 7: could not read, skipping
Pose 8: could not read, skipping
Pose 9: could not read, skipping

Summary (pose, rmsd Å):
1 nan
2 nan
3 nan
4 nan
5 nan
6 nan
7 nan
8 nan
9 nan


[11:58:57] Explicit valence for atom # 1 N, 4, is greater than permitted
[11:58:57] ERROR: Could not sanitize molecule ending on line 42
[11:58:57] ERROR: Explicit valence for atom # 1 N, 4, is greater than permitted
[11:58:57] Explicit valence for atom # 1 N, 4, is greater than permitted
[11:58:57] ERROR: Could not sanitize molecule ending on line 94
[11:58:57] ERROR: Explicit valence for atom # 1 N, 4, is greater than permitted
[11:58:57] Explicit valence for atom # 1 N, 4, is greater than permitted
[11:58:57] ERROR: Could not sanitize molecule ending on line 146
[11:58:57] ERROR: Explicit valence for atom # 1 N, 4, is greater than permitted
[11:58:57] Explicit valence for atom # 1 N, 4, is greater than permitted
[11:58:57] ERROR: Could not sanitize molecule ending on line 198
[11:58:57] ERROR: Explicit valence for atom # 1 N, 4, is greater than permitted
[11:58:57] Explicit valence for atom # 1 N, 4, is greater than permitted
[11:58:57] ERROR: Could not sanitize molecule ending on l

In [ ]:
visualize_docked_molecule(top_ligand_indices[2])

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
compute_rmsd(ligand_index = top_ligand_indices[2])

Pose 1: RMSD (MCS-based) = 0.562 Å (using 7 atoms)
Pose 2: RMSD (MCS-based) = 0.565 Å (using 7 atoms)
Pose 3: RMSD (MCS-based) = 0.540 Å (using 7 atoms)
Pose 4: RMSD (MCS-based) = 0.540 Å (using 7 atoms)
Pose 5: RMSD (MCS-based) = 0.538 Å (using 7 atoms)
Pose 6: RMSD (MCS-based) = 0.538 Å (using 7 atoms)
Pose 7: RMSD (MCS-based) = 0.565 Å (using 7 atoms)
Pose 8: RMSD (MCS-based) = 0.565 Å (using 7 atoms)
Pose 9: RMSD (MCS-based) = 0.565 Å (using 7 atoms)

Summary (pose, rmsd Å):
1 0.5616648436209911
2 0.5648892770256375
3 0.539716272528438
4 0.5396633822857515
5 0.5377909793365843
6 0.5376149159220174
7 0.5649039811754831
8 0.5647543968222648
9 0.5647070908927149


In [ ]:
visualize_docked_molecule(top_ligand_indices[3])

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
compute_rmsd(ligand_index = top_ligand_indices[3])

Pose 1: RMSD (MCS-based) = 0.247 Å (using 6 atoms)
Pose 2: RMSD (MCS-based) = 0.251 Å (using 6 atoms)
Pose 3: RMSD (MCS-based) = 0.247 Å (using 6 atoms)
Pose 4: RMSD (MCS-based) = 0.251 Å (using 6 atoms)
Pose 5: RMSD (MCS-based) = 0.251 Å (using 6 atoms)
Pose 6: RMSD (MCS-based) = 0.251 Å (using 6 atoms)
Pose 7: RMSD (MCS-based) = 0.251 Å (using 6 atoms)
Pose 8: RMSD (MCS-based) = 0.251 Å (using 6 atoms)
Pose 9: RMSD (MCS-based) = 0.247 Å (using 6 atoms)

Summary (pose, rmsd Å):
1 0.24738168702781718
2 0.25148572528349733
3 0.24735711085770057
4 0.2513367213847874
5 0.2512926479095045
6 0.2514829508927177
7 0.25131317307197343
8 0.2513887633273077
9 0.24748875805769294


In [ ]:
visualize_docked_molecule(top_ligand_indices[4])

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
compute_rmsd(ligand_index = top_ligand_indices[4])

Pose 1: RMSD (MCS-based) = 0.514 Å (using 11 atoms)
Pose 2: RMSD (MCS-based) = 0.514 Å (using 11 atoms)
Pose 3: RMSD (MCS-based) = 0.514 Å (using 11 atoms)
Pose 4: RMSD (MCS-based) = 0.513 Å (using 11 atoms)
Pose 5: RMSD (MCS-based) = 0.514 Å (using 11 atoms)
Pose 6: RMSD (MCS-based) = 0.514 Å (using 11 atoms)
Pose 7: RMSD (MCS-based) = 0.514 Å (using 11 atoms)
Pose 8: RMSD (MCS-based) = 0.514 Å (using 11 atoms)
Pose 9: RMSD (MCS-based) = 0.514 Å (using 11 atoms)

Summary (pose, rmsd Å):
1 0.5137623744557985
2 0.5136309644141615
3 0.5135302045102837
4 0.5134766606729771
5 0.5136071298090535
6 0.5137014734677122
7 0.5136814833107349
8 0.5137127519819513
9 0.5137090169416118
